In [14]:
# @-*- coding = utf-8 _*_
# @Time : 2021/4/28 20:35
# @Author : 杨昌军
# @File : textmining.ipynb
# @Software : PyCharm

In [1]:
import sys #导入sys模块，以实现在指定地方结束程序
import pandas as pd
import numpy as np
import jieba
import jieba.posseg as pseg
import jieba.analyse
import re
from collections import Counter  #导入collections模块中的counter动作以进行词频统计。
from gensim.models.word2vec import Word2Vec
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

from a_libs.y_folder import get_this_file_parent_folder as get_pf,select_folder_path as slc_pth



e:\pycharmprojects\demo0410\venv\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
def excel_to_list(excel_file,col_name):  #定义一个函数，将excel中的某一列转化为list列表。
    comments_text_dataframe = pd.read_excel(excel_file, converters={
        'id': str,'text':str})  # 读取excel_file所指称的文件，存入dataframe中，并将id列内容转换为字符串格式（此前为数值格式）。此时comments_text_dataframe是个dataframe。
    comments_text_dataframe = comments_text_dataframe.drop_duplicates()  #全部数据去重。
    comments_text_dataframe.index += 1  #让comments_text_dataframe的索引号+1，即原来是从0开始，现在从1开始。
    comment_list = comments_text_dataframe['text'].values.tolist()
    return comment_list,comments_text_dataframe  #返回一个包含指定列数据的列表、一个包含导入的excel全部内容的dataframe、指定列的列名。
def filter_sentence(data_frame,data_list,col_name,filter_word): #定义一个函数，筛选包含指定词语的句子。
    data_filtered_frame = data_frame.loc[data_frame[col_name].str.contains(filter_word, na=False)] #在text列中查找有“生活”字符串的数据。
    data_filtered_frame.index = range(len(data_filtered_frame))
    data_filtered_frame.index += 1
    comment_filtered_list = []
    for screen in data_list:
        screen = str(screen)
        if screen.find(filter_word) != -1:
            comment_filtered_list.append(screen)
    return data_filtered_frame,comment_filtered_list
def save_all_filtered_data_to_excel(data_frame,save_filename):  #定义一个函数，将筛选过的数据存入excel。
    if save_filename == '':
        save_filename = (input("请输入你存储筛选过的数据的Excel文件名，不用输入后缀名“.xlsx”：")).strip()
        print(f"你准备将筛选过的数据存入{save_filename}.xlsx文件中，请选择此文件的保存路径。")
        save_path = slc_pth(get_pf()) #选择保存路径。
    data_frame.to_excel(f'{save_path}\\{save_filename}.xlsx',index=False)
    print(f'\033[5;30;47m…………筛选后的数据已保存至\033[1;46m{save_path}\033[0m下的“{save_filename}.xlsx”中…………\033[0m')
def clean_text(comments_list,regular_expression): #定义一个函数，进行文本清理。
    text_str = ''.join(str(comments_list))  #通过.join方法合并。
    pattern = re.compile(f'{regular_expression}')
    text_str = re.sub(pattern,'',text_str)
    return text_str
def word_segment_and_pos_tagging(new_words, stop_words, text_str):  #定义一个函数，先加载自定义词典，再加载停用词词典，然后对输入的字符串text_str进行分词，最终输出为列表。
    jieba.enable_paddle()  #利用飞桨Paddle深度学习框架，训练序列标注（双向GRU）网络模型实现分词及词性标注。
    jieba.load_userdict(new_words)    #加载自定义词典。
    stopwords = [line.strip() for line in open(stop_words, encoding='UTF-8').readlines()]   #加载停用词词典，并创建一个停用词列表。此时，stopwords是一个列表。
    print("正在利用飞桨paddle模式对文本进行分词和词性标注，需要一定时间，请稍等。")
    seg_and_tagging_all_word_list = pseg.lcut(text_str,use_paddle=True) #分词并标注词性，默认开启飞桨paddle模式下的分词与词性标注功能，因为对比过开与不开的效果，感觉确实开后分词与标注更精确，具体如何还要研究。
    segged_and_tagged_word_pair_list = []  #既包含词又包含词性的列表。
    for tagging_word in seg_and_tagging_all_word_list:
        if tagging_word.word not in stopwords:  #去除停用词，tagging_word.word为取tagging_word这个pair元组里word的值。
            segged_and_tagged_word_pair_list.append(tagging_word)
    segged_word_list = []  #只包含词，不包含词性。
    for seged_and_tagged_word in segged_and_tagged_word_pair_list:
        seged_word = seged_and_tagged_word.word
        segged_word_list.append(seged_word)
    return  segged_and_tagged_word_pair_list,segged_word_list  #segged_and_tagged_word_pair_list是一个既包含词又包含词性的list，segged_word_list仅包含词，不包含词性。
def sort_word_frequency(data):  #此函数用于统计词频并进行排序。返回的是一个排好序的列表。
    word_frequency_dic = dict(Counter(data))  # 用Counter()统计列表word_list中的词频，并转化为字典word_frequency_dic。注意，这里的Counter的首字母必须大写，否则会出错。
    word_frequency_dic_sorted = sorted(word_frequency_dic.items(), key=lambda x: x[1],reverse=True)  # [:100] #这里是先用字典的.items()方法将字典转换为列表，然后用列表的sorted方法对列表中的元素进行排序，后面的都是参数。
    return word_frequency_dic_sorted
def show_the_highest_ranked_word(data): #此函数用于直接显示排序最高的单词，可以不用。
    num = int(input("你想显示前多少个词频最高的单词/词语："))
    for i in range(num):
        word, count = sort_word_frequency(data)[i]
        print("{0:<10}{1:>5}".format(word, count))  #{0:<10}{1:>5}指定变量值的输出格式，其含义为输出第一个（0）变量时，左对齐（<），最小宽度为10，第二个（1）右对齐（>），最小宽度为5。
def line_word_cut(data_list,new_words,stop_words,re_expression):
    jieba.enable_paddle()
    print("正在利用飞桨paddle模式对单条文本进行分词，需要一定时间，请稍等。")
    jieba.load_userdict(new_words)    #加载自定义词典。
    stopwords = [line.strip() for line in open(stop_words, encoding='UTF-8').readlines()]
    line_word_cut_list = []
    for line in data_list:
        pattern = re.compile(f'{re_expression}')
        text_str = re.sub(pattern,'',line)
        line = pseg.cut(text_str,use_paddle=True)
        word_list = []
        for word_pair in line:  #line为分好词的每一条评论。
            if word_pair.word not in stopwords:
                word_list.append(word_pair.word)
        line_word_cut_list.append(word_list)
    return line_word_cut_list

In [3]:
excel_file = 'E:\\PycharmProjects\\Demo0410\\work_data_liziqi\\liziqi_total_comments.xlsx' #可以直接填写要处理的excel文件的绝对路径。
col_name = 'text'  #可以直接填写此excel文件中要处理的数据所在的列名。
filter_word = '生活'  #筛选词
new_words = 'E:\\PycharmProjects\\Demo0410\\work_data_liziqi\\new_words.txt'    #自定义词典。
stop_words = 'E:\\PycharmProjects\\Demo0410\\work_data_liziqi\\stop_words.txt'    #停用词词典。
stop_words_after_filtered = 'E:\\PycharmProjects\\Demo0410\\work_data_liziqi\\stop_words_after_filtered.txt'  #相当于上面的停用词词典，主要是加了“生活”一词。
re_expression = '\n|\[.{1,8}\]|[^\u4e00-\u9fa5]'

In [4]:
comment_list, comment_dataframe = excel_to_list(excel_file, col_name)  #调用一开始定义的excel_to_list函数。注意，此函数会返回两个值，一个是包含导入的excel指定列数据的列表，一个是包含导入的excel全部数据的dataframe。
comment_filtered_dataframe,comment_filtered_list = filter_sentence(comment_dataframe,comment_list,col_name,filter_word)

In [64]:
cleaned_txt = clean_text(comment_filtered_list, re_expression)
segged_and_tagged_word_pair_list,segged_word_list = word_segment_and_pos_tagging(new_words, stop_words_after_filtered, cleaned_txt)
#……………………准备工作，分词和词性标注完成…………………………

Paddle enabled successfully......


正在利用飞桨paddle模式对文本进行分词和词性标注，需要一定时间，请稍等。


In [66]:
jieba.load_userdict(new_words)  #加载新词词典。
jieba.analyse.set_stop_words(stop_words_after_filtered) #加载停用词典。
tf_idf_comment_filtered_text = ''.join(str(comment_filtered_list))  #将过滤后的语料连接成字符串。
tf_idf_re_expression = '\[.{1,8}\]'  #表示表情包的正则式。
tf_idf_comment_text = clean_text(tf_idf_comment_filtered_text,tf_idf_re_expression)  #去掉表情包里的字符。
jieba.analyse.set_idf_path('idf.txt.big')  #使用从jieba下载的自定义逆向文件频率（IDF）语料库，经验证，关键词提取明显更准确。
tf_idf_word = jieba.analyse.extract_tags(tf_idf_comment_text,topK=20,withWeight=True,allowPOS=('ns','nr', 'n', 'nt','nw','nz','vn', 'vd','v','a','ad','an','d','m','q')) #参数allowPOS=为仅包括指定词性的词，这里主要是包括各种名词、动词、形容词、副词、量词。
#进行主题抽取。

In [67]:
line_word_cut_list = line_word_cut(comment_filtered_list,new_words,stop_words,re_expression)  #将每一条评论进行分词，输出的是list of list。
comment_filtered_dataframe['cut'] = line_word_cut_list #将分割好的词加进dataframe中。
comment_filtered_dataframe = comment_filtered_dataframe[['text','cut','name','time','count','id']]  #重新安排dataframe各列的顺序。

n_dim = 200  #word2vec词向量的维度。
model = Word2Vec(vector_size=n_dim, window=5, min_count=1)  #
model.build_vocab(comment_filtered_dataframe.cut)  #生成词表，注意这里的comment_filtered_dataframe.cut必须为list of list，否则会出错。
%time model.train(comment_filtered_dataframe.cut,total_examples=w2vmodel.corpus_count,epochs=10)  #在评论集上建模。
model.save('comment.mod') #保存模型。

Paddle enabled successfully......


正在利用飞桨paddle模式对单条文本进行分词，需要一定时间，请稍等。


In [70]:
list_vec = []
for sentence in line_word_cut_list:
    word_vec  = 0
    for word in sentence:
       word_vec  += model.wv[word]
    sentence_vec = word_vec/len(word)
    list_vec.append(sentence_vec)
comment_filtered_dataframe['vec'] = list_vec

In [71]:
estimator = KMeans(n_clusters=10)
estimator.fit(list_vec)
label_pred = estimator.labels_

In [93]:
current_dataframe = pd.DataFrame(comment_filtered_dataframe,columns = ['text','cut','cluster'])
current_dataframe.to_excel('排序.xlsx')

In [112]:
for i in range(len(current_dataframe['text'])):
    i += 1
    if current_dataframe.loc[i,'cluster'] == 0:
        print(current_dataframe.loc[i,'text'])


有人说有团队是包装，可即便有团队，又有几个人，能日复一日的忍受劳作的繁琐，一如既往坚持本心。有毅志，知美，倔强而好强，却以温柔对抗生活的艰辛与琐碎，日复一日，终于活出自己的随心所欲。
太能干了 ，全能的李子柒什么都会 ，向往这样的生活，等我们年龄大了也要回农村生活去
农村生活就像世外桃源一样 平淡美好，没有城市的繁忙喧嚣
能把朴素的田园生活拍的那么唯美有意境，是多少人向往的生活，看完真的很治愈～[爱慕]
唯美的画面，没有繁华都市的喧嚣与浮躁，宁静而温馨的生活状态，向往和憧憬[微笑]
看李子柒的视频就是一种享受 能把农村的田园生活过得如诗似画般美 独一无二的女人[赞][666][666][666]
看过视频后总是充满对生活的热爱，对质朴的追求，还有心灵的宁静
每次看李子柒的视频就特别想回农村种田过田园般生活
只要你够聪明和勤奋，也能够把看似枯燥乏味的生活经营得像童话般的美好。这就是李子柒给世人展示的生活本应该过的样子。这背后的辛苦和付出也许只有她自己知道。美好的生活是来之不易的。
非常喜欢李子柒的视频！我也喜欢这样的生活！安静、宁静，没有喧闹。踏踏实实地陪好自己的家人！
与世无争的田园生活，喧嚣的城市生活的人们的向往
这像一朵花一样的是什么青菜，绿绿的，人人羡慕把生活过成了诗的李子柒
天啊，真是把生活过成了诗
太美了，每次看李子柒的视频都觉得过的是神仙般的生活，出门就是世外桃源的感觉
我特喜欢吃这些，说真的这才是生活，
李子柒是个能文能武的奇女子，每次看到子柒的视频都觉得生活真美好，眼前的一地鸡毛也都变得不重要了
李子柒活出了中国女人最真实的记录，还有世人眼中向往的田园生活。
神仙般的生活，把日子过成诗，赞
美食生活的榜样者，愿人们都学会，享受美食的乐趣
太喜欢了，红色的衣服，绿色的植物，与大自然融为一体。构成一幅唯美的画。这简直就是诗般生活！
向往美好的生活，看见你的视频我好开心，美好的一天。
很羡慕你能把生活过成诗[送心]
太美了，景美，食美，人更美。这或许就是向往的生活吧。
一米一饭，当思来之不易；一丝一缕，恒念物力维艰。农村的生活并没有李子柒视频里的效果，反倒多了一份艰辛和劳累，只有富人才会思念农村生活。
像陶渊明诗歌里的田园生活。很佩服李子柒也很喜欢她不娇柔不做作，很多时候干活不带手套，很真实。
李子柒姑娘，你怎么什么都会做啊，城市里的阿姨真羡慕大山里的生活

In [109]:
current_dataframe.loc[1,'cluster']

'现在的明星那么多导演编剧陪着也未必拍得出她的这种效果。这确实是她真正的生活，她只是用更美的意境来呈现而已。更唯美的手段来呈现而已，我不懂为什么老是有人喷她。'

In [96]:
framegrp = current_dataframe.groupby('cluster')


In [97]:
framecls = framegrp.agg(sum)
cuttxt = lambda x: " ".join(m_cut(x))
chapclsres= framecls.text.apply(cuttxt)
chapclsres

In [99]:
x0 = list_vec[label_pred == 0]
x1 = list_vec[label_pred == 1]
x2 = list_vec[label_pred == 2]
plt.scatter(x0[:, 0], x0[:, 1], c = "red", marker='o', label='label0')
plt.scatter(x1[:, 0], x1[:, 1], c = "green", marker='*', label='label1')
plt.scatter(x2[:, 0], x2[:, 1], c = "blue", marker='+', label='label2')
plt.xlabel('petal length')
plt.ylabel('petal width')
plt.legend(loc=2)
plt.show()

TypeError: only integer scalar arrays can be converted to a scalar index

In [45]:
 #在评论集上建模。
print(model.wv['美好'].shape)
model.wv['美好']

(300,)


array([ 6.50828183e-02,  5.45614004e-01,  6.61508664e-02,  2.96942532e-01,
       -6.87738582e-02, -5.14309585e-01,  4.89970624e-01,  1.05606949e+00,
       -7.00010285e-02, -1.96761772e-01,  9.75505337e-02, -2.84855545e-01,
       -4.11312766e-02,  9.27759111e-02, -3.35113108e-01, -2.04699725e-01,
        1.99788243e-01, -2.33106576e-02,  8.34386200e-02, -3.03963959e-01,
       -3.78964633e-01,  7.52356425e-02,  1.82356700e-01,  1.12217821e-01,
        3.98389637e-01, -1.31491750e-01, -4.42996860e-01, -6.49553761e-02,
       -4.49640214e-01, -4.76246178e-01,  1.12774149e-01, -2.63571709e-01,
        5.82394674e-02, -1.57202724e-02, -2.39260703e-01,  3.11100960e-01,
        3.58318806e-01, -5.01340032e-01,  1.96804091e-01,  1.30305156e-01,
       -2.57571667e-01, -7.36259914e-04,  1.70108937e-02, -4.30858642e-01,
        3.45495582e-01,  3.54333371e-01,  1.54954344e-01,  4.11755741e-01,
        1.74311828e-02,  5.00393510e-01,  7.94964507e-02,  1.71997771e-01,
       -3.86620164e-01, -

In [55]:
model.wv.most_similar('生活')

[('城市', 0.9998788237571716),
 ('田园', 0.9998723268508911),
 ('这种', 0.9998677968978882),
 ('日子', 0.999847948551178),
 ('现实', 0.9998449683189392),
 ('农村', 0.999841570854187),
 ('喜欢', 0.9998389482498169),
 ('美好', 0.9998385906219482),
 ('地方', 0.9998348355293274),
 ('才是', 0.9998344779014587)]

In [13]:
def pipei(s1 = '',s2 = ''):
    results = []
    k = 0
    for i in s1:
        result = 0
        for j in s2:
            try:
                result = max(model.sinmilarity(i, j), result)
            except KeyError:
                if i == j:
                    result = 1
            results.append(result)
    for kk in results:
        k = k + kk
    return  k/len(s1)
def pp(s1,s2):
    return  (pipei(s1,s2)+pipei(s2,s1))/2
